In [ ]:
import pycountry
import pycountry_convert

In [ ]:
def map_to_continent(df, country_col='country', location_col='location', continent_col='continent'):
    """
    Map countries and locations to continents.

    Parameters:
    df (pandas.DataFrame): DataFrame containing country and location columns
    country_col (str): Name of the country column
    location_col (str): Name of the location column
    continent_col (str): Name of the continent column to fill

    Returns:
    pandas.DataFrame: DataFrame with filled continent values
    """
    df_copy = df.copy()

    # Country to continent mapping using pycountry
    def get_continent_from_country(country_name):
        try:
            country = pycountry.countries.get(name=country_name)
            if country:
                continent_code = pycountry_convert.country_alpha2_to_continent_code(country.alpha_2)
                return pycountry_convert.convert_continent_code_to_continent_name(continent_code)
        except Exception as e:
            print(f"Error mapping country '{country_name}': {e}")
        return None

    # Special location patterns to continent mapping
    location_patterns = {
        'Atlantic': 'Atlantic Ocean',
        'Mid-Atlantic': 'Atlantic Ocean',
        'Indian Ocean': 'Indian Ocean',
        'Pacific': 'Pacific Ocean',
        'Caribbean': 'North America',
        'Antarctic': 'Antarctica',
        'Kermadec': 'Oceania',
        'Fiji': 'Oceania',
        'Tonga': 'Oceania',
        'Vanuatu': 'Oceania',
        'Kuril': 'Asia',
        'Alaska': 'North America',
        'Philippines': 'Asia',
        'Sumatra': 'Asia',
        'Loyalty Islands': 'Oceania',
        'Macquarie': 'Oceania'
    }

    def get_continent(row):
        # If continent is already filled, return it
        if pd.notna(row[continent_col]):
            return row[continent_col]

        # Try to get continent from country first
        if pd.notna(row[country_col]):
            continent = get_continent_from_country(row[country_col])
            if continent:
                return continent

        # If country not found or is null, try location patterns
        if pd.notna(row[location_col]):
            location = str(row[location_col]).lower()
            for pattern, continent in location_patterns.items():
                if pattern.lower() in location:
                    return continent

        return None

    # Apply the mapping
    mask = df_copy[continent_col].isna() & df_copy[country_col].notna()
    if mask.any():
        df_copy.loc[mask, continent_col] = df_copy[mask].apply(get_continent, axis=1)

        # Print statistics
        filled = mask.sum() - df_copy[continent_col].isna().sum()
        print(f"Filled {filled} out of {mask.sum()} missing continent values")

        if df_copy[continent_col].isna().sum() > 0:
            print("\nUnmapped locations:")
            unmapped = df_copy[df_copy[continent_col].isna()]
            print(unmapped[[country_col, location_col]].drop_duplicates())

    return df_copy

In [ ]:
df = map_to_continent(df)
df

In [ ]:
df['continent'] = df['continent'].replace('None', np.nan)
df.isna().sum()